# Setup






In [ ]:
!pip install tensorflow-addons
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import tensorflow_addons as tfa

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 8.3 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


# Prepare the data

In [ ]:

from google.colab import drive
drive.mount('/content/drive/')

# 작업 경로를 my drive로 변경하여 구글 드라이브 접속 후 작업 경로 확인
print('현재 작업 경로 :', os.getcwd())
os.chdir('/content/drive/MyDrive/')
print('변경된 작업 경로 :', os.getcwd())

# 이미지 데이터를 저장한 폴더 경로
data_folder = "/content/drive/MyDrive/dataset"

# 이미지를 로드하고 NumPy 배열로 변환하는 함수
def load_and_preprocess_image(image_path, target_size=(128, 128)):
    img = load_img(image_path, target_size=target_size)
    img_array = img_to_array(img)
    img_array /= 255.0  # 이미지를 [0, 1] 범위로 정규화
    return img_array



# 데이터 생성
data = []

# 폴더 탐색
for root, dirs, files in os.walk(data_folder):
    for file in files:
        if file.endswith((".jpg", ".jpeg", ".png")):
            # 이미지 파일 경로
            image_path = os.path.join(root, file)

            # 레이블 (폴더 이름을 사용할 수도 있음)
            label = os.path.basename(root)

            # 데이터 딕셔너리에 추가
            data.append({"image_path": image_path, "label": label})


images = []
labels = []

for sample in data:
    image_path = sample["image_path"]
    label = sample["label"]

    img_array = load_and_preprocess_image(image_path)
    images.append(img_array)
    labels.append(label)


# 이미지 데이터를 NumPy 배열로 변환
images = np.array(images)
labels = np.array(labels)

# 레이블을 숫자로 인코딩
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

# 레이블을 원-핫 인코딩
num_classes = len(np.unique(encoded_labels))
one_hot_labels = tf.keras.utils.to_categorical(encoded_labels, num_classes)


# 훈련 및 테스트 세트로 데이터 분할
train_images, test_images, train_labels, test_labels = train_test_split(
    images, one_hot_labels, test_size=0.2, random_state=42
)

# 데이터 형태 확인
print("Train images shape:", train_images.shape)
print("Train labels shape:", train_labels.shape)
print("Test images shape:", test_images.shape)
print("Test labels shape:", test_labels.shape)

Mounted at /content/drive/
현재 작업 경로 : /content
변경된 작업 경로 : /content/drive/MyDrive
Train images shape: (556, 128, 128, 3)
Train labels shape: (556, 2)
Test images shape: (140, 128, 128, 3)
Test labels shape: (140, 2)


# Modeling 1

In [ ]:
# MLP 모델 정의

from keras.layers import BatchNormalization
from keras.regularizers import l2
from keras.callbacks import EarlyStopping

# Assuming your input shape is (img_height, img_width, num_channels)
input_shape = (128, 128, 3)

model = keras.Sequential([
    keras.layers.Flatten(input_shape=input_shape),
    BatchNormalization(),  # Add Batch Normalization before the activation function
    keras.layers.Dense(512, activation='relu', kernel_regularizer=l2(0.01)),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(256, activation='relu', kernel_regularizer=l2(0.01)),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(128, activation='relu', kernel_regularizer=l2(0.01)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(64, activation='relu', kernel_regularizer=l2(0.01)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(32, activation='relu', kernel_regularizer=l2(0.01)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(2, activation='softmax')
])

# 모델 컴파일
optimizer = keras.optimizers.Adam(learning_rate=0.0003)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# 모델 요약 출력
model.summary()

model.fit(train_images, train_labels, epochs=100, batch_size=10)
model.evaluate(test_images, test_labels)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 49152)             0         
                                                                 
 batch_normalization_1 (Bat  (None, 49152)             196608    
 chNormalization)                                                
                                                                 
 dense_6 (Dense)             (None, 512)               25166336  
                                                                 
 dropout_5 (Dropout)         (None, 512)               0         
                                                                 
 dense_7 (Dense)             (None, 256)               131328    
                                                                 
 dropout_6 (Dropout)         (None, 256)               0         
                                                      

[1.6858874559402466, 0.5142857432365417]

In [ ]:
model.evaluate(test_images, test_labels)

5/5 [==============================] - 0s 55ms/step - loss: 1.6859 - accuracy: 0.5143


[1.6858874559402466, 0.5142857432365417]

# Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

# 테스트 데이터에 대한 예측값 생성
predictions = model.predict(test_images)

# 예측값을 클래스로 변환
predicted_classes = np.argmax(predictions, axis=1)

# 실제 레이블을 클래스로 변환
true_classes = np.argmax(test_labels, axis=1)

# Confusion matrix 계산
cm = confusion_matrix(true_classes, predicted_classes)

# TP, FN, FP, TN 출력
print("\nConfusion Matrix:")
print("================================")
print("                Predicted Positive | Predicted Negative")
print("--------------------------------|-------------------")
print(f"Actual Positive | {cm[1,1]:<16} | {cm[1,0]:<16}")
print("--------------------------------|-------------------")
print(f"Actual Negative | {cm[0,1]:<16} | {cm[0,0]:<16}")
print("================================")

# True Positive (TP)
tp = cm[1, 1]

# False Positive (FP)
fp = cm[0, 1]

# True Negative (TN)
tn = cm[0, 0]

# False Negative (FN)
fn = cm[1, 0]

# True Positive Rate (Sensitivity, Recall, TPR)
tpr = tp / (tp + fn)

# False Positive Rate (FPR)
fpr = fp / (fp + tn)

# True Negative Rate (Specificity, TNR)
tnr = tn / (tn + fp)

# False Negative Rate (FNR)
fnr = fn / (fn + tp)

# 출력
print("\nPerformance Metrics:")
print("===============================")
print(f"True Positive Rate (TPR): {tpr}")
print(f"False Positive Rate (FPR): {fpr}")
print(f"True Negative Rate (TNR): {tnr}")
print(f"False Negative Rate (FNR): {fnr}")
print("===============================")

# classification report 출력
print("Confusion Matrix:")
print(cm)

# classification report 출력
print("\nClassification Report:")
print(classification_report(true_classes, predicted_classes))

5/5 [==============================] - 0s 82ms/step

Confusion Matrix:
                Predicted Positive | Predicted Negative
--------------------------------|-------------------
Actual Positive | 11               | 67              
--------------------------------|-------------------
Actual Negative | 1                | 61              

Performance Metrics:
True Positive Rate (TPR): 0.14102564102564102
False Positive Rate (FPR): 0.016129032258064516
True Negative Rate (TNR): 0.9838709677419355
False Negative Rate (FNR): 0.8589743589743589
Confusion Matrix:
[[61  1]
 [67 11]]

Classification Report:
              precision    recall  f1-score   support

           0       0.48      0.98      0.64        62
           1       0.92      0.14      0.24        78

    accuracy                           0.51       140
   macro avg       0.70      0.56      0.44       140
weighted avg       0.72      0.51      0.42       140

